# Lecture 6.2: Naive Bayes & Decision Trees

This lecture, we are going to train and compare a Gaussian naive bayes model and a decision tree on a real dataset.

**Learning goals:**
- train a Gaussian naive bayes model
- train a decision tree classifier
- visualize and compare the model decision boundaries
- analyse the effect of regularization parameters `max_depth` & `min_samples_leaf`

## 1. Introduction

Let's try to improve our fake banknote detector from lecture 6.1. 🕵️‍♀️ We'll use the same [banknote authentication dataset](https://archive.ics.uci.edu/ml/datasets/banknote+authentication), and try to solve the fake/genuine classification task.

## 2. Classification

### 2.1 Data Munging

Let's load our `.csv` into a pandas `DataFrame`, and have a look at the dataset:

In [ ]:
import pandas as pd

df = pd.read_csv('bank_note.csv')

df.head()

In [ ]:
df.describe()

Recall that we are dealing with 4 features, and one binary label. The features are standardized, so no further preprocessing is necessary.

We can create our feature matrix, `X`, and our label vector, `y`:

In [ ]:
X = df[['feature_2', 'feature_4']].values
y = df['is_fake'].values

And we can visualize the dataset to remember the complexity of the classification task:

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(5,5), dpi=120)
ax = fig.add_subplot()

scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k', alpha=0.5)
ax.set_xlabel('feature_2')
ax.set_ylabel('feature_4')
ax.set_title('Banknote Classification')
handles, labels = scatter.legend_elements()
ax.legend(handles=handles, labels=['genuine', 'fake']);

The data is _not separable_ , and the relationship between `feature_2` and `feature_4` is _non-linear_. This should make a good challenge for our decision trees! 🌳

### 2.2 Training

#### 2.2.1 Gaussian Naive Bayes

We can use our favourite sklearn model api with `.fit` and `.predict()`. The class for Gaussian Naive Bayes is ... `GaussianNB` 😏

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb_clf = GaussianNB()
nb_clf = nb_clf.fit(X, y)

🧠 Can you list all the steps that sklearn had to go through to train this Gaussian Naive Bayes model with the function `.fit()`? 

Let's investigate our fitted _model parameters_.

$ P(y|X) = \frac{P(y) \prod_{i} P(x_{i}|y)}{P(X)}$

where:

$P(x_{i}|y) \sim \mathcal{N}(\mu_{y},\,\sigma_{y}^{2})\,$,

so that:

$P(x_i \mid y) = \frac{1}{\sqrt{2\pi\sigma^2_y}} \exp\left(-\frac{(x_i - \mu_y)^2}{2\sigma^2_y}\right)$

Remember that our Gaussian Naive Bayes model has learned a $\mu$ and a $\sigma$ independently for each feature, and for each class! In this case, we expect $2 x 2$ values. 

⚠️ Sklearn makes things a little confusing (which is out of character), and names $\mu$ -> `theta_`

In [ ]:
nb_clf.theta_

In [ ]:
nb_clf.sigma_

We can also access the class priors, $P(y)$:

In [ ]:
nb_clf.class_prior_

We stated earlier that the class priors are just the relative frequencies of the classes in the dataset... we can check this directly:

In [ ]:
y.mean()

Indeed the mean of `1` values in the dataset matches the $P(y=1)$! 

🧠 These are all the model parameters that the Gaussian Naive Bayes model needed to learn from the dataset to be able to estimate $P(y|X)$. Make sure you understand how that works!

Since we'll be visualizing a lot of classifications in 2D throughout this notebook, let's write some helper functions (code from the [sklearn documentation](https://scikit-learn.org/0.18/auto_examples/svm/plot_iris.html)).

Just like last lecture, the function `.plot_classification()` plots both the dataset and the decision boundary for a given feature matrix `X`, label vector `y`, and a classifier `clf`:

In [ ]:
import numpy as np
from matplotlib.lines import Line2D

def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

def plot_decision_boundary(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    plot_decision_boundary(ax, clf, xx, yy, **params)


def plot_classification(ax, X, y, clf):
    X0, X1 = X[:, 0], X[:, 1]
    xx, yy = make_meshgrid(X0, X1)
    plot_contours(ax, clf, xx, yy,
                      cmap=plt.cm.coolwarm, alpha=0.8)
    scatter = ax.scatter(X0, X1, c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k', alpha=1.0)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    ax.set_title('Bank Notes Classification')
    handles, labels = scatter.legend_elements()
    ax.legend(handles=handles, labels=['genuine', 'fake'])

We can now easily plot our Gaussian Naive Bayes model's predictions 🎨:

In [ ]:
fig = plt.figure(figsize=(5,5), dpi=120)
ax = fig.add_subplot()
plot_classification(ax, X, y, nb_clf) 

Notice the non-linear (quadratic) decision boundary. Does it help classification here? Can you visualise where the Gaussian Distribution means are located?

### 2.2 Training

#### 2.2.1 Decision Trees

We can use our favourite sklearn model api with `.fit` and `.predict()`. The class for decision tree classifiers is ... `DecisionTreeClassifier` 😏

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree_clf = DecisionTreeClassifier(random_state=0)
tree_clf = tree_clf.fit(X, y)

🧠 Can you list all the steps that sklearn had to go through to train this decision tree with the function `.fit()`? 

We usually try to investigate our model parameters after fitting the model. However, decision trees don't have a vector $\theta$ or support vectors, they are _non-parametric models_. In the lecture slides, we introduced decision trees as nested if-statements. So instead, we can investigate their _decision node splits_.

Let's check the _depth_ of our decision tree. Remember this is the maximal length of a decision _branch_. We can also output the total number of leaf nodes:

In [ ]:
print(f'This decision tree has depth {tree_clf.get_depth()}, and contains {tree_clf.get_n_leaves()} leaves')

27 decision levels, that's quite a big tree we've grown here! 🌳

Recall that decision trees make predictions by stepping through their decision nodes. By visualizing our tree's nodes, we can interpret its predictions. Decision trees are sometimes called white box models, because we can examine their inner workings.

We'll use sklearn's `.plot_tree()` method to visualize the decision nodes. We must _truncate_ the tree with a `max_depth=2`, or else the visualization will be too big to fit on the screen.

In [ ]:
from sklearn.tree import plot_tree

fig = plt.figure(dpi=200)
ax = fig.add_subplot(111)
plot_tree(ax=ax, decision_tree=tree_clf, filled=True, max_depth=2, feature_names=['x1', 'x2'], rounded=True, precision=2);

This visualization is packed with information 🤤

- The first line of each node defines its _split_. i.e The feature and the value which partitions incoming data into its children nodes.

- The second line indicates the _gini impurity_ metric associated with each split on the training dataset. Remember, low gini impurity implies homogeneity and is therefore a good thing!

- The third line displays the number of training examples belonging to that node.

- The fourth line shows the _class split_ of this node. i.e How many genuine vs fake bills were present in this node during training. We expect this gap to get larger as we go deeper down the branches.

- The node color represents the same information as the fourth line: bluer nodes contain more genuine bills, redder nodes contain more fake bills. We also expect these hues to get more pronounced closer to the leafs nodes.

Note that all lines except the first tell information specific to _training_. All that is needed for prediction is the feature values of the splits, and the class attribution of each leaf node.

🧠 Take your time to understand this graph and how it was "greedily" built during training.

These splits (if-statements) shape the model's decision boundary. We'd like to visualize this along with the dataset.

We can now easily plot our decision tree classifier's predictions 🎨:

In [ ]:
fig = plt.figure(figsize=(5,5), dpi=120)
ax = fig.add_subplot()
plot_classification(ax, X, y, tree_clf)

That's a funky decision boundary! 😳 It is made exclusively of vertical and horizontal lines because predictions are made from a succession of if-statements on single features. The very thin rectangular areas are typical of _overfit_ decision trees. They try to fit single data points instead of the underlying patterns of the data.

#### 2.2.3 Comparison

Let's compare these two models with another type of classifier: a RBF kernel SVM (see lecture 6.1).

Just like last lecture, we can write a small loop to visualize these models next to eachother. We'll then train a non-linear SVM to compare with the decision tree and the random forest:

In [ ]:
def compare_classification(X, y, clfs, titles):
    fig = plt.figure(figsize=(14, 4), dpi=100)
    for i, clf in enumerate(clfs):
        ax = fig.add_subplot(1, len(clfs), i+1)
        plot_classification(ax, X, y, clf)
        ax.set_title(titles[i])
        
        
from sklearn.svm import SVC

svm_rbf = SVC(kernel='rbf', random_state=0)
svm_rbf = svm_rbf.fit(X, y)

compare_classification(X, y, [svm_rbf, nb_clf, tree_clf], ['RBF SVM', 'Naive Bayes', 'Decision Tree'])

All three models exhibit non-linear decision boundaries, but in very different ways. 
- the SVM is busy maximizing the margin shaped by its distance features, creating smooth "blobs" that follow the edges of the data
- the naive bayes fits its quadratice decision boundary
- the decision tree tries its best to split the dataset with single feature thresholds, splitting the data into nested rectangles


🧠🧠 None of these models seem to be able to correctly predict the fake examples near $[1, -1]$. Why is that?

🧠🧠 In your opinion, which is the algorithm most adapted to this dataset and classification task? Why?

### 2.3 Prediction

Let's test our models by asking them to predict a banknote in the small `genuine` cluster on the left hand side of the graphs above.  We'll use $feature\_1 = -1; feature\_2 = 0$:

In [ ]:
x_predict = np.array([-1, 0]).reshape(1, 2)
print(f'Features: {x_predict}')

svm_rbf_prediction = svm_rbf.predict(x_predict)
print(f'RBF SVM prediction: {svm_rbf_prediction}')

tree_clf_prediction = nb_clf.predict(x_predict)
print(f'Naive Bayes prediction: {tree_clf_prediction}')

forest_clf_prediction = tree_clf.predict(x_predict)
print(f'Decision Tree prediction: {forest_clf_prediction}')

### 2.4 Analysis

#### 2.4.1 Regularization: max depth

We have successfully trained decision trees, but we haven't played with a different regularization hyperparameter: `max_depth`.

`max_depth` "cuts" branches which are too long. i.e During training, nodes deeper than `max_depth` automatically become leaf nodes. 

Let's directly visualize the effect of this hyperparameter on the models's classification by plotting decision boundaries for different values of `max_depth`. We'll be using the handy `**kwargs` as arguments, here's a great [blog post](https://realpython.com/python-kwargs-and-args/) if you haven't heard about them.

In [ ]:
def train_tree(X, y, **kwargs):
    tree_clf = DecisionTreeClassifier(random_state=0, **kwargs)
    return tree_clf.fit(X, y)

max_depth_values = [2, 5, 20]
trees = [train_tree(X, y, max_depth=m) for m in max_depth_values]
titles = [f'max_depth={max_depth}' for max_depth in max_depth_values]

compare_classification(X, y, trees, titles)

We can clearly see the regularizing effect of "cutting off" trees branches after a certain depth. `max_depth` has for effect of reducting the number of nested if-statements, and therefore limiting the number of "angles" in the decision boundary.

## 4. Summary

Today we learned about two new supervised learning models: **naive bayes** and **decision trees**. We started by defining how naive bayes uses Baye's theorem to reframe classification problems as the estimation of a conditional probability. We described the model's tricks, namely: assuming the independence of the features, and assuming that they are Gaussian distributed. We explained how this allows to create fast and simple classifiers only by estimating Gaussian parameters and class priors. We then described the **decision tree** algorithm, and showed how it makes non-linear predictions with nested if-statements. We then went through its training procedure which uses **homogeneity metrics** or **variance reduction** to optimally split decision nodes. After noting these models' tendency to **overfit**, we introduced a **regularization** procedures: changing the trees' maximum depth.
Finally, we applied naive bayes and decision trees to the banknote classification dataset.  We trained and visualized the models, as well as analysing the effect of regularization parameters.

# Resources

## Core Resources


- [sklearn naive bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
- [sklearn decision trees](https://scikit-learn.org/stable/modules/tree.html)  
Official documentation about the tree package, handy breakdown of tree models in sklearn


### Additional Resources

- [Understanding gini impurity](https://victorzhou.com/blog/gini-impurity/)  
Same blog from victor zhou going into the mathematics of gini impurity
- [args and kwargs demystified](https://realpython.com/python-kwargs-and-args/)  
blog post about \*\*kwargs in python
